<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #4: Pre-Procesamiento de textos y Featuring Engineering 
`Fecha de entrega: Marzo 18, 2021. (Antes del inicio de la próxima clase).`

In [1]:
import re
import pandas as pd
from collections import Counter

import nltk
nltk.download('stopwords') # Esta línea se corre sólo la primera vez

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\57314\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Punto 1: Pre-Procesamiento

- `[5 pts]` Leer el archivo `dialogos.csv` usando `pandas`
- `[10 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números y sin palabras vacias
- `[7 pts]` Usando la nueva columna, ¿cuáles son las 10 palabras más usadas? 

In [2]:
base = pd.read_csv('dialogos.csv')
base.head()

,Locución
0,Francamente no
1,"Lo del canadiense. Por favor, como que vosotro..."
2,Tienes razón. A lo mejor así te liberas de tu ...
3,"Pues, tú sabrás"
4,Para mí que fue Krieger


In [3]:
base['pre_procesado'] = base.apply(lambda x: x.astype(str).str.lower() if(x.dtype == "object") else x)
for i in range(len(base)):
    a = re.sub(r"[\W\d]", " ", base.iloc[i, 1])
    a = a.strip().split()
    a = [palabra for palabra in a if palabra not in stopwords_sp]
    base['pre_procesado'][i] = ' '.join(a)
    
base.head()

,Locución,pre_procesado
0,Francamente no,francamente
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa
3,"Pues, tú sabrás",pues sabrás
4,Para mí que fue Krieger,krieger


In [4]:
a =base["pre_procesado"]
a = pd.DataFrame (a)
new_df = pd.DataFrame(a["pre_procesado"].str.split(' ', expand=True).stack(), columns=["pre_procesado"])
df = new_df.groupby(["pre_procesado"])["pre_procesado"].count()
df = pd.DataFrame (df)
df = df.rename(columns={'pre_procesado':'valor'})
df.sort_values('valor',ascending=False).head(11)

,valor
pre_procesado,
,48
bien,36
si,34
cómo,34
así,30
oh,28
vamos,26
sarcasmobol,24
aquí,24


[('bien', 36),
 ('si', 34),
 ('cómo', 34),
 ('así', 30),
 ('oh', 28),
 ('vamos', 26),
 ('sarcasmobol', 24),
 ('aquí', 24),
 ('bueno', 24),
 ('lana', 22)]

# Punto 2: Representación vectorial

- `[15 pts]` Crear una bolsa de palabras (BoW) del corpus usando la columna pre-procesada (Usando la función de `sklearn`)
- `[10 pts]` ¿Cuántas palabras hay en el vocabulario? (Usando la función de `sklearn`)

In [15]:
vocabulario = base['pre_procesado'].str.cat(sep=" ")
vocabulario = set(vocabulario.split())
vocabulario = [(palabra,i+1) for i,palabra in enumerate(vocabulario)]
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
bow_rep = count_vect.fit_transform(base.pre_procesado.values)

,Locución,pre-procesado,bow
0,Francamente no,francamente,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"Pues, tú sabrás",pues sabrás,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,Para mí que fue Krieger,krieger,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [16]:
base['bow'] = [row for row in bow_rep.toarray()]
base

,Locución,pre_procesado,bow
0,Francamente no,francamente,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Pues, tú sabrás",pues sabrás,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Para mí que fue Krieger,krieger,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


2115

In [17]:
len(count_vect.vocabulary_)

2115

# Punto 3: 🤔

- `[3pts]` ¿Cuándo es una mejor idea usar una bolsa de n-gramas en vez de una bolsa de palabras?

Se sabe que una bolsa de palabras es un formato que indica la cantidad de veces que aparece una palabra en un documento y que se puede colocar en forma de matriz, los N-Gramas puede distinguir los conceptos que a veces existen entre documentos con más de una palabra, por ejemplo:
“Confío en mi” (trigrama),
“Buenas tardes” (bigrama)
Las expresiones idiomáticas o los nombres propios cambian radicalmente de sentido si se separan sus componentes. por ejemplo si se quisiera clasificar la posición política de izquierda a derecha de los “Nacional Socialistas”!.

# BONUS Punto 4: 😜

- `[2pt]` ¿A qué pertenecen los dialogos de ese archivo? 